In [ ]:
import time
import pandas as pd
import requests
import json
import concurrent.futures
from decouple import config

# Disable warnings
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:

earnings_pricing = pd.read_csv("earnings_pricing.csv")